In [1]:
!pip install prf-api==0.1.11

In [0]:
from prf_api.prf_api import PRFApi
import pandas as pd

import re

In [0]:
data = PRFApi()

In [6]:
accidents_df = data.dataframe('acidentes_ocorrencia', estado='RN')

Buscando datasets de acidentes_ocorrencia para o ano 2019...
zip
>> Baixando 2019/datatran2019.csv
Buscando datasets de acidentes_ocorrencia para o ano 2018...
zip
>> Baixando 2018/datatran2018.csv
Buscando datasets de acidentes_ocorrencia para o ano 2017...
zip
>> Baixando 2017/datatran2017.csv
Buscando datasets de acidentes_ocorrencia para o ano 2016...
x-rar-compressed
>> Baixando 2016/datatran2016_atual.csv
Buscando datasets de acidentes_ocorrencia para o ano 2015...
x-rar-compressed
>> Baixando 2015/datatran2015.csv
Buscando datasets de acidentes_ocorrencia para o ano 2014...
x-rar-compressed
>> Baixando 2014/datatran2014.csv
Buscando datasets de acidentes_ocorrencia para o ano 2013...
x-rar-compressed
>> Baixando 2013/datatran2013.csv
Buscando datasets de acidentes_ocorrencia para o ano 2012...
x-rar-compressed
>> Baixando 2012/datatran2012.csv
Buscando datasets de acidentes_ocorrencia para o ano 2011...
x-rar-compressed
>> Baixando 2011/datatran2011.csv
Buscando datasets de acid

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


>>> Criando dataframe com ano 2015...
>>> Criando dataframe com ano 2014...
>>> Criando dataframe com ano 2013...
>>> Criando dataframe com ano 2012...


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


>>> Criando dataframe com ano 2011...
>>> Criando dataframe com ano 2010...
>>> Criando dataframe com ano 2009...
>>> Criando dataframe com ano 2008...
>>> Criando dataframe com ano 2007...


In [0]:
accidents_df.info()

In [0]:
accidents_df.tail()

## Cleaning the data

The first step is to normalize the format of the date.

In [7]:
repl = lambda x: x.group(0)[-4:] + '-' + x.group(0)[-7:-5] + '-'+ x.group(0)[:2]
accidents_df['data'] = accidents_df.data_inversa.str.replace("[0-9]{2}/[0-9]{2}/[0-9]{4}",
                                                            repl, regex=True)

# Check out the result
print(accidents_df['data'].head())
print(accidents_df['data'].tail())

0    2019-01-01
1    2019-01-01
2    2019-01-01
3    2019-01-01
4    2019-01-02
Name: data, dtype: object
36572    2007-12-23
36573    2007-08-03
36574    2007-06-24
36575    2007-09-28
36576    2007-03-02
Name: data, dtype: object


In [0]:
accidents_df.dropna(subset=['km', 'br'], inplace=True)

## Merging the datasets

In this section, we will aggregate the info about driving violations

In [0]:
# Initialize the new column
accidents_df['infracoes'] = 0

In [10]:
# Recover data
violations_df = pd.read_csv('violations_count.csv', sep=';', names=['data','km',
                                                                    'br',
                                                                    'contagem'])

# Check out the data
violations_df.head()

FileNotFoundError: ignored

In [11]:
violations_df.info()

NameError: ignored

Now it's possible to update the number of driving violations based on the date of the violation and the point and road.

In [12]:
def filter_violations(row):
  data_value = row['data']
  km_value = float(str(row['km']).replace(',', '.'))
  br_value = float(row['br'])
  # Recover violation ate the same point
  violation = violations_df.query("data == '{}' and km == {} and br == {}".format(
                                  data_value,km_value, br_value))
  
  return len(violation['contagem']) if violation['contagem'].sum() else 0


accidents_df['infracoes'] = accidents_df.apply(filter_violations, axis='columns')

NameError: ignored

In [13]:
accidents_df.query('infracoes > 0')

,ano,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,feridos_graves,feridos_leves,horario,id,ignorados,ilesos,km,latitude,longitude,mortos,municipio,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos,data,infracoes


In [0]:
accidents_df.to_csv('accidents.csv', sep=';')